In [2]:
import sys
import pandas as pd
import math
import random
from ipynb.fs.full.chapter_1 import *

60039 98409 129189 152283 152354 152411 163207 197028 200160 357976 376771 392723 532935 600085 622755 1065555


In [4]:
# Return the first pattern of length k showing up in all strings of 'dna' with at most d mismatches 

def motif_enumeration(dna: list[str], k: int, d: int) -> list[str]:
    patterns = set()
    for string in dna:
        for i in range(len(string) - k + 1):
            kmer = string[i:i+k]
            neighborhood = neighbors(kmer, d)
            for neighbor in neighborhood:
                if all(
                    any(
                        hamming_distance(neighbor, string[j:j+k]) <= d 
                        for j in range(len(string) - k + 1)
                    )
                    for string in dna
                ):
                    patterns.add(neighbor)
    return list(patterns)


In [5]:
# Return the total edit distance bewtween a pattern and all strings of dna

def distance_between_patterns_and_strings(pattern: str, dna: list[str]):
    k = len(pattern)
    distance = 0
    for text in dna:
        hamming = math.inf
        for i in range(len(text) - k + 1):
            kmer = text[i:i+k]
            if hamming > hamming_distance(pattern, kmer):
                hamming = hamming_distance(pattern, kmer)
        distance += hamming
    return distance  

In [6]:
# Generate all nucleotide combinations of length k

def all_strings(k: int):
    base_string = k * 'A'
    kmers = neighbors(base_string, k)
    return kmers

In [7]:
# Return a kmer with the minimum total edit distances to all strings in dna

def median_string(dna: list[str], k: int):
    distance = math.inf
    median = str
    kmers = all_strings(k)
    for kmer in kmers:
        if distance > distance_between_patterns_and_strings(kmer, dna):
            distance = distance_between_patterns_and_strings(kmer, dna)
            median = kmer
    return median

In [8]:
# Construct a profile matrix given a list of strings

def profile_matrix(dna: list[str]):
    profile = []
    for col in range(len(dna[0])):
        temp_dict = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        for row in range(len(dna)):
            base = dna[row][col]
            temp_dict[base] += (1 / len(dna))
        profile.append(temp_dict)
    return profile

In [9]:
# Calculates the score of a profile, returning the sum of the fraction of non-consensus bases at each position

def score(profile: list[dict[str, float]]):
    total_mismatches = 0
    for col in profile:
        max_value = max(col.values())
        max_base = ''
        for base, count in col.items():
            if count == max_value:
                max_base = base
        for base, count in col.items():
            if base != max_base:
                total_mismatches += count
    
    total_mismatches *= sum(profile[0].values())
    
    return total_mismatches

In [10]:
# Returns the most frequent kmer given a profile

def profile_most_probable_kmer(text: str, k: int,
                               profile: list[dict[str, float]]) -> str:
    min_probability = 0
    frequent_kmer = ''
    for i in range(len(text) - k + 1):
        probability = 1
        window = text[i:i+k]
        for j in range(len(window)):
            probability *= profile[j][window[j]]
        if probability > min_probability:
            min_probability = probability
            frequent_kmer = window

    if min_probability == 0:
        return text[:k]
    return frequent_kmer 

In [11]:
# Greedy algorithm for searching motifs, iterates through dna selecting the motif with the lowest score 

def greedy_motif_search(dna: list[str], k: int, t: int) -> list[str]:
    best_motifs = [string[:k] for string in dna]
    for i in range(len(dna[0]) - k + 1):
        motif1 = dna[0][i:i+k]
        motifs = [motif1]
        for j in range(1, t):
            profile = profile_matrix(motifs)
            motif = profile_most_probable_kmer(dna[j], k, profile)
            motifs.append(motif)
        if score(profile_matrix(motifs)) < score(profile_matrix(best_motifs)):
            best_motifs = motifs
    return best_motifs


In [12]:
# Construct a profile matrix given a list of strings, adding a pseudocount of +1 to all values 

def profile_matrix_pseudocount(dna: list[str]):
    profile = []
    for col in range(len(dna[0])):
        temp_dict = {'A': 1, 'C': 1, 'G': 1, 'T': 1}
        for row in range(len(dna)):
            base = dna[row][col]
            temp_dict[base] += (1 / (len(dna) + 1))
        profile.append(temp_dict)
    return profile

In [13]:
# greedy_motif_search with pseudocounts

def greedy_motif_search_pseudocounts(dna: list[str], k: int, t: int) -> list[str]:
    best_motifs = [string[:k] for string in dna]
    for i in range(len(dna[0]) - k + 1):
        motif1 = dna[0][i:i+k]
        motifs = [motif1]
        for j in range(1, t):
            profile = profile_matrix_pseudocount(motifs)
            motif = profile_most_probable_kmer(dna[j], k, profile)
            motifs.append(motif)
        if score(profile_matrix_pseudocount(motifs)) < score(profile_matrix_pseudocount(best_motifs)):
            best_motifs = motifs
    return best_motifs

In [14]:
# Selects random motifs from strings in dna, iteratively choose the most probable kmer from strings as the new motifs

def randomized_motif_search(dna: list[str], k: int, t: int) -> list[str]:
    motifs = []
    for string in dna:
        pos = random.randint(0, len(string) - k)
        kmer = string[pos:pos+k]
        motifs.append(kmer)
    best_motifs = motifs
    while True:
        profile = profile_matrix_pseudocount(motifs)
        motifs = [profile_most_probable_kmer(string, k, profile) for string in dna]
        if score(profile_matrix_pseudocount(motifs)) < score(profile_matrix_pseudocount(best_motifs)):
            best_motifs = motifs
        else:
            return best_motifs